# **Initial setup**

Install Bambu and required packages:

In [ ]:
!echo "deb http://ppa.launchpad.net/git-core/ppa/ubuntu $(cat /etc/os-release | grep UBUNTU_CODENAME | sed 's/.*=//g') main" >> /etc/apt/sources.list.d/git-core.list
!apt-key adv --keyserver keyserver.ubuntu.com --recv-keys A1715D88E1DF1F24
!apt-get update
!apt-get install -y --no-install-recommends build-essential ca-certificates gcc-multilib git verilator wget
!wget https://release.bambuhls.eu/appimage/bambu-latest.AppImage
!chmod +x bambu-*.AppImage
!ln -sf $PWD/bambu-*.AppImage /bin/bambu
!ln -sf $PWD/bambu-*.AppImage /bin/spider
!ln -sf $PWD/bambu-*.AppImage /bin/tree-panda-gcc
!git clone --depth 1 --filter=blob:none --branch doc/bambu101 --sparse https://github.com/ferrandi/PandA-bambu.git
%cd PandA-bambu
!git sparse-checkout set documentation/bambu101
%cd ..
!mv PandA-bambu/documentation/bambu101/* .

# Integrate external IPs into an HLS generated design
Bambu HLS offers the possibility to integrate RTL modules into the design generated through the High-Level Synthesis flow.
Here we will run through a simple example to show how this is achieved in practice.

## Sample application
A sample application featuring four external IPs will be used.
The file tree is explained in the following to identify each of the necessary bits:

── IP_Integration<br>
&emsp;&emsp;├── module_lib&emsp;&emsp;&emsp;&emsp;&emsp; : External IPs library<br>
&emsp;&emsp;│&emsp;&emsp;├── module_lib.h&emsp; &nbsp; : Library header declaring Verilog IPs C stub interfaces<br>
&emsp;&emsp;│&emsp;&emsp;├── module_lib.xml&ensp; : XML file describing Verilog IPs interfaces<br>
&emsp;&emsp;│&emsp;&emsp;├── module1.v&emsp;&emsp;&ensp; : Verilog IP implementation of module1<br>
&emsp;&emsp;│&emsp;&emsp;├── module2.v&emsp;&emsp;&ensp; : Verilog IP implementation of module2<br>
&emsp;&emsp;│&emsp;&emsp;├── printer1.v&emsp;&emsp;&emsp; : Verilog IP implementation of printer1<br>
&emsp;&emsp;│&emsp;&emsp;├── printer2.v&emsp;&emsp;&emsp; : Verilog IP implementation of printer2<br>
&emsp;&emsp;│&emsp;&emsp;├── module1.c&emsp;&emsp;&ensp; : C stub used to emulate module1 Verilog IP<br>
&emsp;&emsp;│&emsp;&emsp;├── module2.c&emsp;&emsp;&ensp; : C stub used to emulate module2 Verilog IP<br>
&emsp;&emsp;│&emsp;&emsp;├── printer1.c&emsp;&emsp;&emsp; : C stub used to emulate printer1 Verilog IP<br>
&emsp;&emsp;│&emsp;&emsp;└── printer2.c&emsp;&emsp;&emsp; : C stub used to emulate printer2 Verilog IP<br>
&emsp;&emsp;├── test.my_ip.xml&emsp;&emsp;&emsp;&emsp;: XML testbench definition file<br>
&emsp;&emsp;└── top.c&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;: C application to feed into the HLS tool

## XML interface description
The HLS tool needs an XML description of the interface of the external IPs to be integrated into the HLS generation flow.
The user must define each external IP interface specifying module name, I/O ports, other IPs which may be used by the implementation, and the IP implementation.

The definition of the library for current example is available in [module_lib/module_lib.xml](module_lib/module_lib.xml) along with the external IPs implementation files [module_lib/module1.v](module_lib/module1.v), [module_lib/module2.v](module_lib/module2.v), [module_lib/printer1.v](module_lib/printer1.v), [module_lib/printer2.v](module_lib/printer2.v).

## C stub for simulation
Also a C definition of the external IPs is needed along with the Verilog implmenetation and the XML interface description.
This is useful to generate the verification testebench results to be compared with the generated design simulation.

C stub definition for external IPs of this example can be found in [module_lib/module1.c](module_lib/module1.c), [module_lib/module2.c](module_lib/module2.c), [module_lib/printer1.c](module_lib/printer1.c), [module_lib/printer2.c](IP_Integration/module_lib/printer2.c).

## Initializing the HLS tool
To let the HLS tool know about external IPs all previously discussed files must be passed to the tool alongside standard command line options.

The XML interface description is fed as a standard input file.

The external IPs Verilog implementation files are passed through the --file-input-data command line option.

The external IPs C stubs for simulation are passed through the --C-no-parse command line option to be used for simulation only.

In [ ]:
%cd /content/IP_Integration
!bambu top.c --top-fname=my_ip $PWD/module_lib/module_lib.xml --channels-type=MEM_ACC_11 --generate-tb=test.my_ip.xml --simulate --file-input-data=module_lib/module1.v,module_lib/module2.v,module_lib/printer1.v,module_lib/printer2.v --C-no-parse=module_lib/module1.c,module_lib/module2.c,module_lib/printer1.c,module_lib/printer2.c